<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/LlamaIndex_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

At its core, LlamaIndex contains a toolkit designed to easily connect LLM’s with your external data.





1.   Creating and Quering Index
2.   Saving and Loading Index
3.   Customize LLM
4.   Customize Prompt
5.   Customize Embedding


In [ ]:
!pip install llama-index pypdf sentence_transformers -q

By default, we use the OpenAI GPT-3 text-davinci-003 model.

https://gpt-index.readthedocs.io/en/latest/getting_started/installation.html

In [ ]:
import os
import openai
openai.api_key = ""
os.environ["OPENAI_API_KEY"] = ""

##Creating Index

https://gpt-index.readthedocs.io/en/latest/guides/primer/index_guide.html

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader('book').load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("What is this text about?")
print(response)


This text is about the dangers of lusting after money and how it can occupy one's mind and lead to paranoia and fear. It also discusses how Naval Ravikant combined his vocation and avocation to make money in a way that felt like play.


In [ ]:
response = query_engine.query("who is this text about?")
print(response)


This text is about Naval Ravikant.


In [ ]:
response = query_engine.query("when was this book published")
print(response)


This book was published in 2020.


In [ ]:
response = query_engine.query("list 5 important points from this book")
print(response)


1. Understand How Wealth Is Created
2. Find and Build Specific Knowledge
3. Play Long-Term Games with Long-Term People
4. Take on Accountability
5. Build or Buy Equity in a Business


In [ ]:
response = query_engine.query("what naval says about wealth creation")
print(response)


Naval Ravikant says that wealth creation is possible through ethical means. He suggests seeking wealth instead of money or status, and suggests that one should own equity in a business in order to gain financial freedom. He also suggests that one should give society what it wants but does not yet know how to get, and that this should be done at scale. He further states that money is how we transfer wealth, and that wealth is assets that earn while you sleep.


##Saving and Loading Index

In [ ]:
index.storage_context.persist("naval_index")

In [ ]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="naval_index")
# load index
new_index = load_index_from_storage(storage_context)

In [ ]:
new_query_engine = new_index.as_query_engine()
response = new_query_engine.query("who is this text about?")
print(response)


This text is about Naval Ravikant.


##Customizing LLM's

https://gpt-index.readthedocs.io/en/latest/how_to/customization/service_context.html

In [ ]:
from llama_index import LLMPredictor, ServiceContext

from langchain.chat_models import ChatOpenAI

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))


service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)


custom_llm_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

In [ ]:
custom_llm_query_engine = custom_llm_index.as_query_engine()
response = custom_llm_query_engine.query("who is this text about?")
print(response)

This text is about Naval Ravikant.


##Custom Prompt

In [ ]:
from llama_index import Prompt

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question and each answer should start with code word AI Demos: {query_str}\n"
)
qa_template = Prompt(template)

In [ ]:
query_engine = custom_llm_index.as_query_engine(text_qa_template=qa_template)
response = query_engine.query("who is this text about?")
print(response)

AI Demos: This text is about Naval Ravikant.


##Custom Embedding

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

# load in HF embedding model from langchain
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2'))
service_context = ServiceContext.from_defaults(embed_model=embed_model)

new_index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context,
)

In [ ]:
query_engine = new_index.as_query_engine()
response = query_engine.query("list 5 important points from this book")
print(response)


1. Building wealth and being happy are skills that can be learned.
2. The Almanack of Naval Ravikant is a collection of Naval's wisdom and experience from the last ten years.
3. This book provides insight into Naval's principles for building wealth and creating long-term happiness.
4. This book is available for free download in pdf and e-reader versions on Navalmanack.com.
5. Eric Jorgenson is a product strategist and writer who joined the founding team of Zaarly.


In [ ]:
query_engine = new_index.as_query_engine()
response = query_engine.query("what naval says about wealth creation")
print(response)


Naval Ravikant says that wealth creation is not a one-time thing, but a skill that needs to be learned. He suggests asking yourself if what you are doing is authentic to you and if you are productizing, scaling, and using labor, capital, code, or media to do so. He also states that money is a way to transfer wealth, and that wealth is assets that can earn while you sleep, such as businesses, factories, robots, computer programs, and even houses that can be rented out.
